# ECON 326: Interactions and Non-linear Terms in Regressions

## Authors
* Jonathan Graves (jonathan.graves@ubc.ca)
* Devan Rawlings (rawling5@student.ubc.ca)

## Prerequisites
* Multiple regression
* Simple regression
* Data analysis and introduction

## Outcomes
In this worksheet, you will learn:

* How to incorporate interaction terms into a regression analysis
* How to interpret models with interaction terms
* How to create models which include non-linear terms
* How to compute simple marginal effects for models with non-linear terms
* How to explain polynomial regressions as approximations to a non-linear regression function

### Notes

<span id="fn1">[<sup>1</sup>](#fn1s)Data is provided under the Statistics Canada Open License.  Adapted from Statistics Canada, 2016 Census Public Use Microdata File (PUMF). Individuals File, 2020-08-29. This does not constitute an endorsement by Statistics Canada of this product.</span>

<span id="fn2">[<sup>2</sup>](#fn2s)Stargazer package is due to: Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer </span>

In [ ]:
#load the data and set it up

library(tidyverse)
library(haven)
library(car)
library(lmtest)

source("hands_on_tests_5.r")

census_data <- read_dta("02_census2016.dta")
census_data <- as_factor(census_data)
census_data <- census_data %>%
               mutate(lnwages = log(wages))
census_data <- filter(census_data, !is.na(census_data$wages))
census_data <- filter(census_data, !is.na(census_data$mrkinc))
glimpse(census_data)

# Part 1: Interactions in Regression Models

One of the most common extensions to multiple regression models is to include **interaction** terms.  What is an interaction term?  It's basically a term which represents the product of two (or more) variables in a model.

For example, if we have a dummy variable for being an immigrant ($I_i$) and a dummy variable for having a college degree ($D_i$), the _interaction_ of these two variables is the product $D_i \times I_i$.  This can seem complicated but it also has a simple interpretation: it is now a dummy for being _both_ an immigrant and having a college degree.  You can see why this is true:

$$ D_i \times I_i = 1 \iff D_i = 1 \text{ and } I_i = 1$$

This is why these terms are so important for understanding regressions: they provide us with a simple way to describe and study how _combinations_ of our explanatory variables impact our model.  These variables enter into our regression models in exactly the same way as usual:

$$Y_i = \beta_0 + \beta_1 I_i + \beta_2 D_i + \beta_3 D_i \times I_i + \epsilon_i$$

At this point, you can see that this is just a multiple regression model - the only difference is that one of the variables is a combination of the other variables.  From an estimation perspective, there's no issue - you can use OLS to estimate a model with interaction terms, just like normal.  However, as we have seen, there are important differences when it comes to the _interpretation_ of these models.  Let's learn more about this in this worksheet.

There are (in general) two ways to create interactions in R: (i) manually (i.e. creating a new variables which is $D_i \times I_i$ then adding it to the regression), or (ii) using the built-in tools in R.  However, method (i) is a trap!  You should never use this method.  Why?  There are two reasons:

1.  The main reason is that R (and you, the analyst) lose track of the relationship between the created interaction variable and the underlying variables.  This means that you can't use other tools to analyse these relationship (there are many packages such as ``margins`` which allow you to investigate complex interaction) which is a big loss.  You also can't perform post-regression analysis on the underlying variables in a simple way anymore.
2.  The second reason is that it's easy to make mistakes.  You might define the interaction incorrectly (possible!).  However, it's more of an issue if later on you change the underlying variables and then forget to re-compute the interactions.  It also makes your code harder to read.

Bottom line: don't do it.  Interaction in R are easy to create: you simply use the ``:`` or ``*`` operator when defining an interaction term.  

* The ``:`` operator creates the interaction(s) of the two variables in question
* The ``*`` operation creates the interactions(s) _and_ the main effects of the variables as well

Even better: if you are interacting two qualitative (factor) variables, it will automatically "expand" the interaction into every possible combination of the variables.  A lot less work!

For example, let's look at a regression model which interacts immigration status and sex:

In [ ]:
regression1 <- lm(wages ~ immstat + sex + immstat:sex, data = census_data)

#regression1 <- lm(wages ~ immstat*sex, data = census_data) #an alternative way to run the same regression

summary(regression1)

There are a few important things to notice about this regression result.  First, take a close look at the terms:

* ``immstatimmigrants`` this is the main effect for being an immigrant.  You _might_ immediately say that this is the impact of being an immigrant - but this is _not true_.  Why?  Because immigrant shows up in two places!  We have to be a little more careful - this is the effect of being an immigrant for individuals in the base group (female respondents) 
* ``immstatimmigrants:sexmale`` this is the interaction effect of being a man an being an immigrant.  Basically, male immigrants earn -3200 less than female immigrants.

You can see this interpretation in the regression model itself:

$$W_i = \beta_0 + \beta_1 I_i + \beta_2 S_i + \beta_3 I_i \times S_i + \epsilon_i$$

Consider:

$$\frac{\Delta W_i}{\Delta I_i} = \beta_1 + \beta_3 S_i$$

The marginal effect of being an immigrant _changes_ depending on what the value of $S_i$ is!  For women (the level where $S_i = 0$) it's $\beta_1$.  For men (the level where $S_i =1$), it's $\beta_1 + \beta_3$.  This is why, in an interaction model, it doesn't really make sense to talk about the "effect of immigration" - because there isn't a single, immutable effect.  It is different for different people!

You can talk about the _average_ effect, which is just $\beta_1 + \beta_3 \bar{S_i}$ - but that's not really what people are asking about when they are discussing the immigration effect, in general.

This is why it's very important to carefully think about a regression model with interaction terms - the model may seem simple to estimate, but the interpretation is more complex.

## Interactions with Continuous Variables

So far, we have just looked at interacting qualitative variables - but you can interact any types of variables!

* Qualitative-Qualitative
* Qualitative-Quantitative
* Quantitative-Quantitative

The format and syntax in R is similar, with some small exceptions to deal with certain combinations of variables.  However (again), you do need to be careful with interpretation.

For example, let's look at the interaction of wages and sex on market income.  In a regression equation, this would be expressed like:

$$M_i = \beta_0  + \beta_1 W_i + \beta_2 S_i + \beta_3 W_i \times S_i + \epsilon_i$$

Notice that, just like before:

$$\frac{\partial M_i}{\partial W_i} = \beta_1 + \beta_3 S_i$$

There are two _different_ "slope" coefficients; basically, male and female respondents can have a different return to wages.  Let's see this in R:

In [ ]:
regression2 <- lm(mrkinc ~ wages + sex + wages:sex, data = census_data)

summary(regression2)

As we can see here, the male respondents in our model earn about 0.047 dollars more in market income per dollar of wages earned than female respondents.  

This addresses the common problem of estimating a regression model where you think the impact of a continuous variable might be different across the two groups.  One approach would be to run the model only for men, and only for women, the compare - but this isn't a good idea.  Those regressions have a much smaller sample size, and if you have other controls in the model, you will "throw away" information.  The interaction method is much better.

# Part 2: Non-linear Terms in Regression Models

You might have been puzzled by why these models were called "linear" regressions.  The reason is because they are linear in the _coefficients_: the dependent variable is expressed as  a linear combination of the explanatory variables.

This implies that we can use the same methods (OLS) to estimate models that including linear combinations of _non-linear functions_ of the explanatory variables.  We have actually already seen an example of this: remember using ``log`` of a variable?  That's a non-linear function!

As we learned when considering ``log``, the most important difference here is again regarding interpretations, not the actual estimation.

In R, there is one small complicatation: when you want to include mathematical expressions in a model formula, you need to "isolate" then using the ``I()`` function.  This is because many operations in R, like ``+`` or ``*`` have a special meaning in a regression model.

For example, let's consider a quadratic regression - that is, including both $W_i$ (wages) and $W_i^2$ (wages squared) in our model

In [ ]:
regression3 <- lm(mrkinc ~ wages + I(wages^2), data = census_data)

summary(regression3)

As you can see, we get regression results much like we would expect.  However, how do we interpret them?  The issue is that _wages_ enters into two places.  We need to carefully interpret this model, using our knowledge of the equation:

$$M_i = \beta_0 + \beta_1 W_i + \beta_2 W_i^2 + \epsilon_i$$

$$\implies \frac{\partial M_i}{\partial W_i} = \beta_1 + 2 \beta_2 W_i$$

You will notice something special about this; the marginal effect is _non-linear_.  As $W_i$ changes, the effect of wages on $M_i$ changes.  This is because we have estimated a quadratic relationship; the slope of a quadratic changes as the explanatory variable changes.  That's what we're seeing here!

This makes these models relatively difficult to interpret, since the marginal effects change (often dramatically) as the explanatory variables change.  You frequently need to carefully interpret the model and often (to get estimates) perform tests on _combinations_ of coefficients, which can be done using things like the ``car`` package or the ``lincom`` function.  You can also compute this manually, using the formula for the sum of variances.

For example, let's test if the marginal effect of wages is significant at $W_i = \bar{W_i}$.  This is the most frequently reported version of this effects, often called the "marginal effect at the means".

In [ ]:
m <- mean(census_data$wages)

linearHypothesis(regression3, hypothesis.matrix = c(0, 1, 2*m), rhs=0) 

As we can see, it is highly significant

> _Think Deeper_: what is the vector ``c(0, 1, 2*m)`` doing in the above expression?

Let's see exactly what those values are.  Recall the formula:

$$V(aX + bY) = a^2 V(X) + b^2 V(Y) - 2abCov(X,Y)$$

In our situation, $X = Y = W_i$, so this is:

$$V(\beta_1 + 2\bar{W_i}\beta_2) = V(\beta_1) + 4\bar{W_i}^2V(\beta_2) - 2(2\bar{W_i})Cov(\beta_1,\beta_2)$$

Fortunately, these are all things we have from the regression and its variance-covariance matrix:


In [ ]:
v <- vcov(regression3)
coefs <- regression3$coefficients
v

var <- v[2,2] + 4*m*v[3,3] - 4*m*v[3,2]

var

coef <-  coefs[[2]] + 2*m*coefs[[3]]

print("Coefficent Combination and SD")
round(coef,3)
round(sqrt(var),3)

As you can see, this gets fairly technical and is not something you will want to do without a very good reason.  In general, it's a better idea to rely on some of the packages written for R that handle this task for the (specific) model you are interested in evaluating.

### Aside: Why Polynomial Terms?

You might be wondering why econometricians spend so much time talking about models that included polynomial terms, when those are (realistically) a very small set of the universe of possible functions of an explanatory variable (you already know why we talk about ``log`` so much!).  

The reason is actually approximation.  Consider the following _non-linear_ model:

$$Y_i = f(X_i) + e_i$$

This model is truly non-linear (and not just in terms of the parameters).  How can we estimate this model?  It's hard!  There are techniques to estimate complex models like this, but how far can we get with good-old OLS?  The answer is - provided that $f$ is "smooth" - pretty far.

Think back to introductory calculus; you might remember a theorem called _Taylor's Theorem_ (https://en.wikipedia.org/wiki/Taylor%27s_theorem).  It says that a smoothly differentiable function can be arbitrarily well-approximated (about a point) by a polynomial expansion:

$$f(x) = f(a) + f'(a)(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \cdots + \frac{f^{(k)}(a)}{k!}(x-a)^k + R_k(x)$$

and the error term $R_k(x) \to 0$ as $x \to a$ and $k \to \infty$.

Look closely at this expression.  Most of the terms (like $f'(a)$) are constants.  In fact, you can show that this can be written like:

$$f(x) = \beta_0 + \beta_1 x + \beta_2 x^2 + \cdots + \beta_k x^k + r$$

Putting this into our expression above gives us the relationship:

$$Y_i = \beta_0 + \beta_1 X_i + \beta_2 X_i^2 + \cdots + \beta_k X_i^k+ \epsilon_i$$

Which is a linear regression model!  What this say is actually very important: linear regression models can be viewed as _approximations_ to nonlinear regressions, provided we have enough polynomial terms.  This is one complication: the error term is definitely not uncorrelated.  You can learn more about how to address this issue in other courses, but at the most the omitted variable bias is relatively small as $k \to \infty$.

# Part 3: Exercises

In this set of exercises, you will get some hands-on experience with estimating and interpreting non-linear regression models, as well as learning to scrutinize multiple regression models for their limitations. 

## Practical Activity 1

Consider the following regression model:

$$W_i = \beta_1 + \beta_2 I_i + \beta_3 F_i + \beta_4 V_i + \beta_5 I_i\times F_i + \beta_6 I_i \times V_i + \epsilon_i$$

$W_i$ denotes wages, $V_i$ is a factor variable for the worker's visible minority group, $I_i$ is a dummy for immigrant status and $F_i$ is a factor variable for the field of postsecondary education.

### Short Answer 1
<em>Prompt:</em> How should we interpret the coefficients $\beta_5$ and $\beta_6$? Why might these effects be important to estimate?

<font color = "red">Answer here in red</font>

Now, let's estimate the model and interpret it concretely. The code below makes the reference level of ``cip2011`` "no postsecondary certificate, diploma or degree" and ``vismin`` to "not a visible minority":

In [ ]:
#Run this!
census_data$cip2011 <- relevel(census_data$cip2011, ref = "no postsecondary certificate, diploma or degree")
census_data$vismin <- relevel(census_data$vismin, ref = "not a visible minority")

We should also take a look at the levels of ``cip2011``, which is a factor level recording the worker's field of education:

In [ ]:
#Run this!
levels(census_data$cip2011)

Estimate the regression equation given at the beginning of the activity.

<em>Tested Objects:</em> ``reg1``.

In [ ]:
#Quiz 1
reg1 <- lm(????, data = census_data)

summary(reg1)

test_1() #quiz1

### Short Answer 2
<em>Prompt:</em> How do we interpret the coefficient estimate on ``immstatimmigrants:cip201103 humanities ``? Which fields do immigrants appear to be most discriminated in? Which fields are immigrants least dicriminated in? How might we explain this intuitively?

<font color = "red">Answer here in red</font>

### Short Answer 3
<em>Prompt:</em> How do you interpret the coefficient estimate on ``immstatimmigrants:visminchinese``? (Hint: Write out the average wage equations for immigrant workers, ethnically Chinese non-immigrant workers, and ethnically Chinese immigrant workers separately.)

<font color = "red">Answer here in red</font>

For the remainder of this exercise, let's use a subset of ``census_data`` that includes only prime-age (25-64 year-old) workers:

In [ ]:
#Run this!

census_data2 <- subset(census_data, agegrp == "25 to 29 years" | agegrp == "30 to 34 years" | agegrp == "35 to 39 years" | agegrp == "40 to 44 years" | agegrp == "45 to 49 years" | agegrp == "50 to 54 years" | agegrp == "55 to 59 years" | agegrp == "60 to 64 years")

Let's re-introduce the simplified education variable by running the code below:

In [ ]:
#Run this!
census_data2 <- 
        census_data2 %>%
        mutate(educ = case_when(
              hdgree == "no certificate, diploma or degree" ~ "Less than high school",
              hdgree == "secondary (high) school diploma or equivalency certificate" ~ "High school diploma",
              hdgree == "trades certificate or diploma other than certificate of apprenticeship or certificate of qualification" ~ "Some college",
              hdgree == "certificate of apprenticeship or certificate of qualification" ~ "Some college",
              hdgree == "program of 3 months to less than 1 year (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "program of 1 to 2 years (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "program of more than 2 years (college, cegep and other non-university certificates or diplomas)" ~ "Some college",
              hdgree == "university certificate or diploma below bachelor level" ~ "Some college",
              hdgree == "bachelor's degree" ~ "Bachelor's degree",              
              hdgree == "university certificate or diploma above bachelor level" ~ "Graduate school",
              hdgree == "degree in medicine, dentistry, veterinary medicine or optometry" ~ "Graduate school",
              hdgree == "master's degree" ~ "Graduate school",
              hdgree == "earned doctorate" ~ "Graduate school",
              hdgree == "not available" ~ "not available"
              )) %>%
        mutate(educ = as_factor(educ))

census_data2$educ <- relevel(census_data2$educ, ref = "Less than high school") #Set "Less than high school" as default factor level

Further, let's construct a simplified version of ``agegrp``:

In [ ]:
#Run this!
census_data2 <- 
        census_data2 %>%
        mutate(age_decade = case_when(
              agegrp == "25 to 29 years" ~ "20s",
              agegrp == "30 to 34 years" ~ "30s",
              agegrp == "35 to 39 years" ~ "30s",
              agegrp == "40 to 44 years" ~ "40s",
              agegrp == "45 to 49 years" ~ "40s",
              agegrp == "50 to 54 years" ~ "50s",
              agegrp == "60 to 64 years" ~ "60s",
              agegrp == "20 to 24 years" ~ "Some college"
              )) %>%
        mutate(age_decade = as_factor(age_decade))

census_data2$age_decade <- relevel(census_data2$age_decade, ref = "20s") #Set "20s" as default factor level

Suppose that we want to test whether the returns to education increase as workers age. First, replace ``cip2011`` with ``educ`` in the regression equation from before. Then, add terms to the regression equation that allow us to run this test. Then, estimate this new model on ``census_data2``.

<em>Tested Objects:</em> ``reg2``.

In [ ]:
#Quiz 2
reg2 <- lm(????, data = census_data2)

summary(reg2)

test_2() #quiz2

### Short Answer 4
<em>Prompt:</em> Do returns to education increase as workers age? Explain why or why not with reference to the regression estimates.

<font color = "red">Answer here in red</font>

## Practical Activity 2

A topic that many labour economists are concerned with, and one that we have discussed before, is the gender-wage gap. In this activity, we will construct a "difference-in-difference" regression to explore this gap using the data available in the census (unfortunately, the census only asks about sex, not gender).

Suppose that we want to estimate the relationship between age, sex and wages. Within this relationship, we suspect that women earn less than men from the beginning of their working lives, but this gap does not change as workers age.

Estimate a regression model (with no additional control variables) that estimates this relationship using ``census_data``.

<em>Tested Objects:</em> ``reg3A``.

In [ ]:
#Quiz 3
census_data$sex <- relevel(census_data$sex, ref = "male") #Do not change; makes "male" the reference level for sex

reg3A <- lm(???)

summary(reg3A)

test_3() #quiz3

### Short Answer 5
<em>Prompt:</em> What is the relationship between age and wages? Between sex and earnings? Is there a significant wage gap? Why might the regression above not give us the "full picture" of the sex wage gap?

<font color = "red">Answer here in red</font>

Estimate the relationship between wages and age for men and women separately, then compare their returns to age.

<em>Tested objects:</em> ``reg3M`` (for males), ``reg3F`` (for females).

In [ ]:
#Quiz 4
reg3M <- lm(???, data = filter(census_data, ???))
reg3F <- lm(???, data = filter(census_data, sex == "female"))

summary(reg3M)
summary(reg3F)

test_4()
test_5() #quiz4

### Short Answer 6
<em>Prompt:</em> Do these regression estimates support your argument from Short Answer 6? Explain.

<font color = "red">Answer here in red</font>

Add one additional term to the multiple regression from Quiz 3 that accounts for the possibility that the sex wage gap can change as workers age.

<em>Tested Objects:</em> ``reg4``.

In [ ]:
#Quiz 5
reg4 <- lm(???, data = census_data)

summary(reg4)

test_6() #quiz5

### Short Answer 7
<em>Prompt:</em> According to the regression you estimated above, what is the nature of the sex wage gap?

<font color = "red">Answer here in red</font>

## Theoretical Activity 1

Suppose that a team of researchers is interested in the relationship between the price of a popular children's gum brand (let's call it "Jumbo Chewz"). The team conducts a series of surveys over a three-year period where they ask 120 households in a Los Angeles neighborhood to report how many packs of Jumbo Chewz they bought in each quarter of the year. The company that produces Jumbo Chewz is interested in estimating the price elasticity of demand for their gum, so they changed the price of a pack of gum each quarter over this period. This survey is voluntary - the team went door-to-door to gather the data, and people could refuse to participate.

After they compile a dataset from the survey responses, the team estimates this model:

$$Q_i^2 = \alpha_1 + \alpha_2 ln(P_i) + \alpha_3 H_i + \epsilon_i$$

$Q_i$ denotes the quantity of gum packs that household i purchased in a given quarter of a given year. That is, each quarter for a given household is a separate observation. $P_i$ is the price of the pack of gum in the given quarter, and $H_i$ is the household size (in number of people). Note that $\hat{\alpha_2}$ is <em>supposed to be</em> the estimated elasticity of demand.

You join the team as a research advisor - in other words, you get to criticize their project and get paid doing so. Sounds great, but you have a lot of work ahead.

### Short Answer 8
<em>Prompt:</em> Are there any omitted variables that the team should be worried about when estimating the model? Give 2 examples of such variables if so, and explain how each variable's omission could affect the estimated elasticity of demand using the formula that we discussed in class.

<font color = "red">Answer here in red</font>

### Short Answer 9
<em>Prompt:</em> Is there anything wrong with the specification of the regression model? If so, explain how to correct it; if not, explain why the speficiation is correct.

<font color = "red">Answer here in red</font>

### Short Answer 10
<em>Prompt:</em> You overhear a teammate saying that the dataset was "perfectly measured." Are they correct? Use information from the description of the study to support your answer, and discuss what effect (if any) this issue might have on regression estimates.

<font color = "red">Answer here in red</font>

### Short Answer 11
<em>Prompt:</em> Is there any potential for sample selection bias in this study? Explain by referencing specfic aspects of the experiment. What effect might this bias have on the estimated elasticity of demand?

<font color = "red">Answer here in red</font>

### Short Answer 12

<em>Prompt:</em> A member of your team writes in the research report that "this estimated elasticity of demand tells us about the preferences of consumers around the United States." Do you have an issue with this statement? Why or why not?

<font color = "red">Answer here in red</font>